In [1]:
import os
import bs4
import re
import pandas as pd
import numpy as np
import datetime
from collections import Counter

files = os.listdir('Ikon/tmp')

In [2]:
def add_fields(sp, tag, cls):
    
    res = sp.findAll(tag, {'class' : cls})
    if len(res) > 0:
        return res[0].text.strip()
    else:
        return None
    
 
    
all_exhibitions = []   
all_artists = []
all_galleries = []

names_dates = {}
galeries_dates = {}

already_processed = set()

for fn in files:
    
    if 2 == 2:
    #if 'IkOn2015.01.26.Mon-2015.02.01.Sun' in fn:
        
        with open('Ikon/tmp/' + fn) as myfile:
            html = myfile.read()
            soup = bs4.BeautifulSoup(html,"lxml")
            exbhis = soup.findAll('div', {'class' : "row"})
            
            for ex in exbhis:
                
                exhibition = {}
        
                year_m  = add_fields(ex, 'div', 'year-month')
                
                if year_m:
                
                    title = add_fields(ex, 'a',   'title')
                
                    if title not in already_processed:

                        already_processed.add(title)

                        day     = add_fields(ex, 'div', 'day')
                        hour    = add_fields(ex, 'div', 'hour')
                        year_m  = ''.join([s for s in year_m if (s == '.' or s.isdigit())])
                        date    = datetime.datetime.strptime(year_m + ' ' + day + ' ' + hour, '%Y.%m. %d %H:%M')
                        gallery = add_fields(ex, 'a',   'gallery')

                        exhibition['date']    = date
                        exhibition['title']   = title
                        exhibition['gallery'] = gallery
                        artists               = ex.findAll('', {'class' : 'artists'}) 
                        exhibition['artists'] =  '; '.join([a.text.replace(';' , ' ') for a in artists])
                        all_exhibitions.append(exhibition)

                        if gallery not in galeries_dates:
                            galeries_dates[gallery] = []
                        galeries_dates[gallery].append(date)
                        

                        all_artists += [a.text for a in artists]
                        
                        for a in artists:
                            if a.text not in names_dates:
                                names_dates[a.text] = []
                            names_dates[a.text].append(date)
                        
                        all_galleries.append(gallery)
                    

df = pd.DataFrame(all_exhibitions)                    

## Saving the outputs

### Top names

In [3]:
for ind, (a, b) in enumerate(Counter(all_artists).most_common(20)):
    print(ind ,'\t', a, '\t',b)

0 	 -, 	 2984
1 	 - 	 2002
2 	 Koronczi Endre, 	 149
3 	 művészettörténész 	 135
4 	 Nádler István, 	 131
5 	 Maurer Dóra, 	 126
6 	 Szombathy Bálint 	 126
7 	 Sugár János, 	 125
8 	 Bukta Imre, 	 123
9 	 Koroknai Zsolt, 	 119
10 	 Kerekes Gábor, 	 115
11 	 Csontó Lajos, 	 115
12 	 Gerhes Gábor, 	 115
13 	 Győrffy László, 	 114
14 	 Török Tamás, 	 114
15 	 Mélyi József 	 111
16 	 --- 	 110
17 	 Bánki Ákos, 	 110
18 	 Szirtes János, 	 110
19 	 Horváth Judit, 	 109


### Saving name & dates, counts

In [3]:
fout = open('Ikon/extracted/all_artists.dat', 'w')
for ind, a in enumerate(all_artists_s):
    fout.write(a.replace('\n', ' ').strip()+'\n')
fout.close()

len(all_artists)

108212

In [17]:
### ALL NAMES
df.to_csv('Ikon/extracted/ikon_exhibitions_raw.csv', sep = '\t',encoding='utf-8')

fout = open('Ikon/extracted/all_artists_raw.dat', 'w')
all_artists_s = sorted(list(set(all_artists)))
for ind, a in enumerate(all_artists_s):
    fout.write(a.replace('\n', ' ').strip()+'\n')
fout.close()


fout = open('Ikon/extracted/all_galleries_raw.dat', 'w')
all_galleries_s = sorted(list(set(all_galleries)))
for a in all_galleries_s:
    fout.write(a.strip()+'\n')
fout.close()

len([line.strip() for line in open('Ikon/extracted/all_artists_raw.dat')])


27498

In [19]:
### COUNT NAMES

fout = open('Ikon/extracted/all_artists_cnt.dat', 'w')
artists_cnt = Counter(all_artists)
for (a, c) in dict(artists_cnt).items():
    fout.write(a.strip() + '\t' + str(c) +'\n')
fout.close()

fout = open('Ikon/extracted/all_galleries_cnt.dat', 'w')
galleries_cnt = Counter(all_galleries)
for a, c in dict(galleries_cnt).items():
    fout.write(a.strip() + '\t' + str(c) +'\n')
fout.close()


len(artists_cnt)

27498

In [13]:
fout = open('Ikon/extracted/names_min_max_date.dat', 'w')
for n, dates in names_dates.items():
    if len(n) > 0 and len(dates) > 0:
        fout.write(n + '\t' + str(min(dates)) + '\t' + str(max(dates)) + '\n' )
fout.close()

In [5]:
folderout = 'extracted'
if not os.path.exists(folderout):
    os.makedirs(folderout)
    
df = df.drop_duplicates()    
    
len(df)

13265

In [9]:
fout = open('Ikon/extracted/all_artists_dates.dat', 'w')

for name, dates in names_dates.items():
    if len(name) > 1:
        fout.write(name + '\t\t' + '\t'.join([str(_) for _ in dates]) + '\n')
fout.close()


fout = open('Ikon/extracted/all_galleries_dates.dat', 'w')
for name, dates in galeries_dates.items():
    fout.write(name + '\t\t' + '\t'.join([str(_) for _ in dates]) + '\n')
fout.close()

27498


27498

In [8]:
len(list(set(all_artists))), ind

(27498, 27497)

## Galleries

In [10]:
df = pd.DataFrame(Counter(all_galleries).most_common(), columns = ['name', 'frequency'])
df.index = df.name
df = df.drop(columns= ['name'])

df.to_csv('gallery_names_raw.csv')